1. LOADING DEPENDICIES.

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
summarizer = pipeline('summarization',model="facebook/bart-large-cnn")

Device set to use cuda:0


2. SCRAPPING THE DATA FROM THE ONLINE BLOG POST 

In [4]:
url1 = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
url2 = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
url3=  "https://writingcenter.unc.edu/tips-and-tools/literature-reviews/"

In [5]:
req = requests.get(url3)

In [6]:
req.text

'\n<!DOCTYPE html>\n<html class="no-js" lang="en-US">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-6b53be0d94491620b86",applicationID:"781537153"};;/*! For license information please see nr-loader-rum-1.279.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Object.keys(r).length?e:r;for(let a in o)if(void 0!==e[a])try{if(null===e[a]){r[a]=null;continue}Array.isArray(e[a])&&Array.isArray(t[a])?r[a]=Array.from(new Set([...e[a],...t[a]])):"object"==typeof e[a]&&"object"==typeof t[a]?r[a]=i(e[a],t[a]):r[a]=e[a]}catch(e){(0,n.R)(1,e)}return r}catch(

In [7]:
soup = BeautifulSoup(req.text,'html.parser')
results = soup.find_all(['h1','p'])

In [8]:
results

[<h1>
 	Literature Reviews	</h1>,
 <p>This handout will explain what literature reviews are and offer insights into the form and construction of literature reviews in the humanities, social sciences, and sciences.</p>,
 <p>OK. You’ve got to write a literature review. You dust off a novel and a book of poetry, settle down in your chair, and get ready to issue a “thumbs up” or “thumbs down” as you leaf through the pages. “Literature review” done. Right?</p>,
 <p>Wrong! The “literature” of a literature review refers to any collection of materials on a topic, not necessarily the great literary texts of the world. “Literature” could be anything from a set of government pamphlets on British colonial methods in Africa to scholarly articles on the treatment of a torn ACL. And a review does not necessarily mean that your reader wants you to give your personal opinion on whether or not you liked these sources.</p>,
 <p>A literature review discusses published information in a particular subject a

In [9]:
soup


<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-6b53be0d94491620b86",applicationID:"781537153"};;/*! For license information please see nr-loader-rum-1.279.0.min.js.LICENSE.txt */
(()=>{var e,t,r={122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Object.keys(r).length?e:r;for(let a in o)if(void 0!==e[a])try{if(null===e[a]){r[a]=null;continue}Array.isArray(e[a])&&Array.isArray(t[a])?r[a]=Array.from(new Set([...e[a],...t[a]])):"object"==typeof e[a]&&"object"==typeof t[a]?r[a]=i(e[a],t[a]):r[a]=e[a]}catch(e){(0,n.R)(1,e)}return r}catch(e){(0,n.

In [10]:
text = [result.text for result in results]
article = ' '.join(text)

In [11]:
text

['\n\tLiterature Reviews\t',
 'This handout will explain what literature reviews are and offer insights into the form and construction of literature reviews in the humanities, social sciences, and sciences.',
 'OK. You’ve got to write a literature review. You dust off a novel and a book of poetry, settle down in your chair, and get ready to issue a “thumbs up” or “thumbs down” as you leaf through the pages. “Literature review” done. Right?',
 'Wrong! The “literature” of a literature review refers to any collection of materials on a topic, not necessarily the great literary texts of the world. “Literature” could be anything from a set of government pamphlets on British colonial methods in Africa to scholarly articles on the treatment of a torn ACL. And a review does not necessarily mean that your reader wants you to give your personal opinion on whether or not you liked these sources.',
 'A literature review discusses published information in a particular subject area, and sometimes inf

In [12]:
article

'\n\tLiterature Reviews\t This handout will explain what literature reviews are and offer insights into the form and construction of literature reviews in the humanities, social sciences, and sciences. OK. You’ve got to write a literature review. You dust off a novel and a book of poetry, settle down in your chair, and get ready to issue a “thumbs up” or “thumbs down” as you leaf through the pages. “Literature review” done. Right? Wrong! The “literature” of a literature review refers to any collection of materials on a topic, not necessarily the great literary texts of the world. “Literature” could be anything from a set of government pamphlets on British colonial methods in Africa to scholarly articles on the treatment of a torn ACL. And a review does not necessarily mean that your reader wants you to give your personal opinion on whether or not you liked these sources. A literature review discusses published information in a particular subject area, and sometimes information in a par

In [13]:
article = article.replace('.','.<eos>')
article = article.replace('!','!<eos>')
article = article.replace('?','?<eos>')

sentences = article.split('<eos>')

In [14]:
sentences

['\n\tLiterature Reviews\t This handout will explain what literature reviews are and offer insights into the form and construction of literature reviews in the humanities, social sciences, and sciences.',
 ' OK.',
 ' You’ve got to write a literature review.',
 ' You dust off a novel and a book of poetry, settle down in your chair, and get ready to issue a “thumbs up” or “thumbs down” as you leaf through the pages.',
 ' “Literature review” done.',
 ' Right?',
 ' Wrong!',
 ' The “literature” of a literature review refers to any collection of materials on a topic, not necessarily the great literary texts of the world.',
 ' “Literature” could be anything from a set of government pamphlets on British colonial methods in Africa to scholarly articles on the treatment of a torn ACL.',
 ' And a review does not necessarily mean that your reader wants you to give your personal opinion on whether or not you liked these sources.',
 ' A literature review discusses published information in a particul

3. CHUNKING THE DATA

In [15]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk+1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [16]:
chunks

[['\n\tLiterature',
  'Reviews\t',
  'This',
  'handout',
  'will',
  'explain',
  'what',
  'literature',
  'reviews',
  'are',
  'and',
  'offer',
  'insights',
  'into',
  'the',
  'form',
  'and',
  'construction',
  'of',
  'literature',
  'reviews',
  'in',
  'the',
  'humanities,',
  'social',
  'sciences,',
  'and',
  'sciences.',
  '',
  'OK.',
  '',
  'You’ve',
  'got',
  'to',
  'write',
  'a',
  'literature',
  'review.',
  '',
  'You',
  'dust',
  'off',
  'a',
  'novel',
  'and',
  'a',
  'book',
  'of',
  'poetry,',
  'settle',
  'down',
  'in',
  'your',
  'chair,',
  'and',
  'get',
  'ready',
  'to',
  'issue',
  'a',
  '“thumbs',
  'up”',
  'or',
  '“thumbs',
  'down”',
  'as',
  'you',
  'leaf',
  'through',
  'the',
  'pages.',
  '',
  '“Literature',
  'review”',
  'done.',
  '',
  'Right?',
  '',
  'Wrong!',
  '',
  'The',
  '“literature”',
  'of',
  'a',
  'literature',
  'review',
  'refers',
  'to',
  'any',
  'collection',
  'of',
  'materials',
  'on',
  'a',

In [17]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [18]:
chunk_id

5

In [19]:
chunks[0]

'\n\tLiterature Reviews\t This handout will explain what literature reviews are and offer insights into the form and construction of literature reviews in the humanities, social sciences, and sciences.  OK.  You’ve got to write a literature review.  You dust off a novel and a book of poetry, settle down in your chair, and get ready to issue a “thumbs up” or “thumbs down” as you leaf through the pages.  “Literature review” done.  Right?  Wrong!  The “literature” of a literature review refers to any collection of materials on a topic, not necessarily the great literary texts of the world.  “Literature” could be anything from a set of government pamphlets on British colonial methods in Africa to scholarly articles on the treatment of a torn ACL.  And a review does not necessarily mean that your reader wants you to give your personal opinion on whether or not you liked these sources.  A literature review discusses published information in a particular subject area, and sometimes informatio

In [20]:
len(chunks[0].split(' '))

495

4. SUMMARIZING THE DATA USING TRANSFORMERS PIPELINE.

In [21]:
res = summarizer(chunks,max_length=120,min_length=30,do_sample=False)

In [22]:
res

[{'summary_text': 'A literature review discusses published information in a particular subject area. The focus of a literature review is to summarize and synthesize the arguments and ideas of others. Literature reviews are written occasionally in the humanities, but mostly in the sciences and social sciences.'},
 {'summary_text': 'If your assignment is not very specific, seek clarification from your instructor. Look for other literature reviews in your area of interest or in the discipline. There are hundreds or even thousands of articles and books on most areas of study.'},
 {'summary_text': 'Cultural studies scholars are accepting popular media as a subject worthy of academic consideration. A literature review must contain at least three basic elements: an introduction or background information section, the body of the review containing the discussion of sources, and a conclusion and/or recommendations section.'},
 {'summary_text': 'There are a few guidelines you should follow during

In [23]:
' '.join([summ['summary_text'] for summ in res])

'A literature review discusses published information in a particular subject area. The focus of a literature review is to summarize and synthesize the arguments and ideas of others. Literature reviews are written occasionally in the humanities, but mostly in the sciences and social sciences. If your assignment is not very specific, seek clarification from your instructor. Look for other literature reviews in your area of interest or in the discipline. There are hundreds or even thousands of articles and books on most areas of study. Cultural studies scholars are accepting popular media as a subject worthy of academic consideration. A literature review must contain at least three basic elements: an introduction or background information section, the body of the review containing the discussion of sources, and a conclusion and/or recommendations section. There are a few guidelines you should follow during the writing stage as well. A literature review in this sense is just like any other

5. CONVERT THE SUMMARY INTO .TXT FILE TO READ.

In [24]:
texts = ' '.join([summ['summary_text'] for summ in res])


In [25]:
with open('literaturesummary.txt','w') as f:
    f.write(texts)